---
title: How long do I descend
description: Finding how many steps it takes for gradient descent to converge
author: Hariom  Narang
date: 'January 6, 2026'
image: images/gradient-descent-tumble-meme.webp
---

In this article, I'm going to investigate how many steps gradient descent takes in linear regression, for a randomly chosen starting point.   
I'm assuming a good starting point is very useful while training AI models, although until now, I've generally started from random weights.  For now, this is an attempt to understand the relationship between the starting weights and the number of iterations gradient descent takes to converge.  
This article does this analysis for the simplest case of linear regression. We assume prior knowledge of linear regression and basic college math.  


## Experiment 

Given:
- starting weight `w0`
- sample of points `(x,y)`
- a learning rate `L`

Find:
How many iterations does gradient descent take to converge?  
We do it for the special case of modelling `y=mx` (linear regression) for now   

Strategy:
Find the equation for `loss` at `k`th iteration in terms of the parameters given above (mainly, we want to model it on the starting weight) 


> The more general question here is modelling how gradient descent (or any other kind of descent) converges for arbitrary functions, inputs and outputs.   

## Classic Linear Regression

I'm defining the classic and the simplest linear regression problem. I assume you know linear regression though.  

Given a set of points $\{(x_0, y_0), (x_1, y_1), \dots, (x_n, y_n)\}$, we want to find a function $y = mx$ which minimizes the mean squared error (called $loss$ here, which we want to minimize)   
$$ loss = \sum_{i=1}^{n} (y_i - mx_i)^2  $$

### Gradient Descent

We start by choosing a random $m = w = w_0$.   
At each step we update $w$ to a new value with the formula as below:  
$$ G = \frac{d}{dw}loss \;=\; \sum_{i=1}^{n} 2x_i(wx_i - y_i) $$
$$ w' = w -  lG \;=\; w - 2l\sum_{i=1}^{n} x_i(wx_i - y_i) $$
$$ w' = w - L\sum_{i=1}^{n} x_i(wx_i - y_i) \qquad (L := 2l) $$

> The substitution $L = 2l$ is simply changing one constant to another (the $2$ is not very useful)

Let's start by expanding the equations for the next weight calculation $w'$ and the $loss$ function

$$ w' = w - L\sum_{i=1}^{n} (wx_i^2 - x_iy_i) \;=\; w - Lw\sum_{i=1}^{n} x_i^2  + L\sum_{i=1}^{n} x_iy_i $$
$$ 
loss = \sum_{i=1}^{n} (y_i - wx_i)^2  \;=\; 
   \sum_{i=1}^{n}(y_i^2 + w^2x_i^2 - 2y_ixi_w) \;=\; 
   \sum_{i=1}^{n} y_i^2 + \sum_{i=1}^{n}w^2x_i^2 - \sum_{i=1}^{n}2wx_iy_i
$$


The terms $\sum_{i=1}^{n} x_i^2 \;, \sum_{i=1}^{n} x_iy_i \;, \sum_{i=1}^{n} y_i^2$ are constants for a given sample of points.   
They will also keep coming everywhere, we will for now substitute them with new symbols.  
$$ \alpha =  \sum_{i=1}^{n} x_i^2 \quad \beta = \sum_{i=1}^{n} x_iy_i \quad \gamma = \sum_{i=1}^{n} y_i^2 $$

This gives us 
$$ w' = w - Lw\alpha + L\beta $$
$$ loss = \gamma + w^2\alpha - 2w\beta $$


## Loss at $k^{th}$ step
The goal is to model the loss function in terms of the weight $w_0$ (the weight that we start gradient descent from) and the step $k$ of gradient descent.  
I will use $w$ to denote $w_0$ (it's easier to write equations, it comes up everywhere)

The loss after the first step ($k = 1$) in gradient descent would be 
$$ loss_1 = \gamma + w_1^2\alpha - 2w_1\beta $$
$$ \text{where} \; w_1 = w - Lw\alpha + L\beta $$

Substitute $w_1$ in loss
$$ loss_1 = \gamma + (w - Lw\alpha + L\beta)^2\alpha - 2(w - Lw\alpha + L\beta)\beta $$

This is quite a lot of work (which was done in a notebook before).  
We have to do this expansion successively for different $k$ values.  
It's a lot of manual work and it's best to use code for this.  
I'll be using `sympy` to expand polynomials now.  

In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# define all variables

x = IndexedBase('x')
y = IndexedBase('y')
n = symbols('n')
i = symbols('i', cls=Idx)


alpha, beta, gamma = symbols('alpha beta gamma')
w, L, k = symbols('w L k')

sum_rg = (i,0,n)

max_k = 15

Our core equations using `sympy`.  
`sympy` works on math equations. I'll make it do polynomial expansion (and also modelling the final expression as a polynomial with $w$ as a free variable)   

The functions below are used for generating these equations

In [3]:
# given w, express w at next step
def next_w(w_expr):
    return -L*alpha*w_expr + L*beta + w_expr

# express loss for given w expression
def loss_at_w(w_expr):
    return alpha*w_expr**2 - 2*beta*w_expr + gamma

# make it a polynomial
def Pw(exp):
    return Poly(exp, w)

# return as an expression, useful for showing results
def E(poly):
    return poly.as_expr()

In [4]:
# show w1 in the form of w0
w_1 = next_w(w)
display(w_1)

# loss given a w0
loss_0 = loss_at_w(w)
display(E(Pw(loss_0)))

# loss at w1
loss_1 = loss_at_w(w_1)
display(E(Pw(loss_1)))

-L*alpha*w + L*beta + w

alpha*w**2 - 2*beta*w + gamma

L**2*alpha*beta**2 - 2*L*beta**2 + gamma + w**2*(L**2*alpha**3 - 2*L*alpha**2 + alpha) + w*(-2*L**2*alpha**2*beta + 4*L*alpha*beta - 2*beta)

We will use `sympy` to the equations for the first few iterations.  
The cell below stores all expressions for deriving $w_k$ given $w = w_0$ in the variable `w_at_k`

In [5]:
def compute_all_ws(current_w, n=max_k):
    r = []
    wi = current_w
    for i in range(n):
        r.append(wi)
        wi = next_w(wi)
    return r


w_at_k = compute_all_ws(w)

# print the first three steps
W = IndexedBase('W')
for i in range(3):
    display(Eq(W[i], w_at_k[i]))

Eq(W[0], w)

Eq(W[1], -L*alpha*w + L*beta + w)

Eq(W[2], -L*alpha*w - L*alpha*(-L*alpha*w + L*beta + w) + 2*L*beta + w)

Calculate the loss expressions using `w_at_k` and store in `losses`

In [6]:
def compute_losses(ws):
    return [loss_at_w(w) for w in ws]

losses = compute_losses(w_at_k)
LOSS = IndexedBase('Loss')
for i in range(3):
    display(Eq(LOSS[i], E(Pw(losses[i]))))

Eq(Loss[0], alpha*w**2 - 2*beta*w + gamma)

Eq(Loss[1], L**2*alpha*beta**2 - 2*L*beta**2 + gamma + w**2*(L**2*alpha**3 - 2*L*alpha**2 + alpha) + w*(-2*L**2*alpha**2*beta + 4*L*alpha*beta - 2*beta))

Eq(Loss[2], L**4*alpha**3*beta**2 - 4*L**3*alpha**2*beta**2 + 6*L**2*alpha*beta**2 - 4*L*beta**2 + gamma + w**2*(L**4*alpha**5 - 4*L**3*alpha**4 + 6*L**2*alpha**3 - 4*L*alpha**2 + alpha) + w*(-2*L**4*alpha**4*beta + 8*L**3*alpha**3*beta - 12*L**2*alpha**2*beta + 8*L*alpha*beta - 2*beta))

## Factorizing

It's time to factorize these equations, or find a pattern in the function for loss.  
The loss function is a quadratic polynomial in terms of $w$ (the starting weight) for all $k$.   

My strategy is to just expand all polynomials for many $k$ values and try to figure out a pattern.  
After that, I will computationally verify if our simplifications are correct for some $k$ values.  

### Finding coefficient of $w^2$ 
Let's start with the coefficient for $w^2$  

I will be writing the coefficients of $w^2$ now for the first few iterations

In [7]:
for i, l in enumerate(losses[:5]):
    term = Pw(l).coeff_monomial(w*w)
    term = cancel(term / alpha)
    COEFF = IndexedBase("Coeff")
    display(Eq(COEFF[i], term * alpha))

Eq(Coeff[0], alpha)

Eq(Coeff[1], alpha*(L**2*alpha**2 - 2*L*alpha + 1))

Eq(Coeff[2], alpha*(L**4*alpha**4 - 4*L**3*alpha**3 + 6*L**2*alpha**2 - 4*L*alpha + 1))

Eq(Coeff[3], alpha*(L**6*alpha**6 - 6*L**5*alpha**5 + 15*L**4*alpha**4 - 20*L**3*alpha**3 + 15*L**2*alpha**2 - 6*L*alpha + 1))

Eq(Coeff[4], alpha*(L**8*alpha**8 - 8*L**7*alpha**7 + 28*L**6*alpha**6 - 56*L**5*alpha**5 + 70*L**4*alpha**4 - 56*L**3*alpha**3 + 28*L**2*alpha**2 - 8*L*alpha + 1))

The pattern is actually pretty simple, the coefficient is of the form

$$ Coeff_k = \alpha(L\alpha - 1)^{2k} $$ 

### Finding coefficient of $w$

In [8]:
for i, l in enumerate(losses[:5]):
    term = Pw(l).coeff_monomial(w)
    term = cancel(term / (-2*beta))
    COEFF = IndexedBase("Coeff")
    display(Eq(COEFF[i], term * (-2*beta)))

Eq(Coeff[0], -2*beta)

Eq(Coeff[1], -2*beta*(L**2*alpha**2 - 2*L*alpha + 1))

Eq(Coeff[2], -2*beta*(L**4*alpha**4 - 4*L**3*alpha**3 + 6*L**2*alpha**2 - 4*L*alpha + 1))

Eq(Coeff[3], -2*beta*(L**6*alpha**6 - 6*L**5*alpha**5 + 15*L**4*alpha**4 - 20*L**3*alpha**3 + 15*L**2*alpha**2 - 6*L*alpha + 1))

Eq(Coeff[4], -2*beta*(L**8*alpha**8 - 8*L**7*alpha**7 + 28*L**6*alpha**6 - 56*L**5*alpha**5 + 70*L**4*alpha**4 - 56*L**3*alpha**3 + 28*L**2*alpha**2 - 8*L*alpha + 1))

Luckily this pattern is also of the form $(L\alpha - 1)^{2k}$

$$ Coeff_k = -2\beta(L\alpha-1)^{2k} $$

### Finding the constant term

The final constant coefficient is, unfortunately, not as simple as the above coefficients. It took some experimentation from my side.   
I'm adding the method that gave me the final simplified form. I used a mixture of solving manually and using `sympy` to find this expression.  
Let's first print it

In [9]:
for i, l in enumerate(losses[:4]):
    term = Pw(l).coeff_monomial(1)
    term = cancel((term - gamma) / (L*beta*beta))
    COEFF = IndexedBase("Coeff")
    display(Eq(COEFF[i], L*beta*beta*term + gamma))

Eq(Coeff[0], gamma)

Eq(Coeff[1], L*beta**2*(L*alpha - 2) + gamma)

Eq(Coeff[2], L*beta**2*(L**3*alpha**3 - 4*L**2*alpha**2 + 6*L*alpha - 4) + gamma)

Eq(Coeff[3], L*beta**2*(L**5*alpha**5 - 6*L**4*alpha**4 + 15*L**3*alpha**3 - 20*L**2*alpha**2 + 15*L*alpha - 6) + gamma)

The sub-expression inside $L\beta^2$ does look very similar to $(L\alpha - 1)^{2k}$. The coefficients are jumbled though.  
I tried writing this sub-expression as a form of $(L\alpha - 1)^{2k}$. After some experimentation, I got to the correct form the sub-expression takes.  
I'll show the form for $Coeff_2$ directly. 

$$
\begin{aligned}
Sub_2 
&= (L^{3} \alpha^{3}  - 4 L^{2} \alpha^{2} + 6 L \alpha - 4 ) \\
&= (L^{3} \alpha^{3} - 3 L^{2} \alpha^{2} + 3 L \alpha - 1) - (L^2\alpha^2 - 3L\alpha + 3)  \\
&= (L^{3} \alpha^{3} - 3 L^{2} \alpha^{2} + 3 L \alpha - 1) - (L^2\alpha^2 - 2L\alpha + 1) + (L\alpha - 2)  \\
&= (L^{3} \alpha^{3} - 3 L^{2} \alpha^{2} + 3 L \alpha - 1) - (L^2\alpha^2 - 2L\alpha + 1) + (L\alpha - 1) - 1  \\
&=  (L\alpha - 1)^3 - (L\alpha - 1)^2 + (L\alpha - 1) - 1 \\
&= -\left[-(L\alpha - 1)^3 + (L\alpha - 1)^2 - (L\alpha - 1) + 1\right] \\
\end{aligned}
$$


The last expression is a geometric series with respect to $(L\alpha -1)$, with $a=1$and$r=-(L\alpha - 1)$. The last power is $ 2k - 1$  
The [sum](https://math.stackexchange.com/questions/4255628/proof-of-geometric-series-formula) of a finite geometric series
$$ x^n + \; ... \; + x^2 + x + 1 = \frac{(1 - x^{n+1})}{(1-x)}  $$
For us $x := -(L\alpha - 1)$ and $n = 2k-1$.   

Calculating the sum
$$ S \;=\; \frac{1 - (1-L\alpha)^{2k}}{1 - (1-L\alpha)} \;=\;  
\frac{1 - (1-L\alpha)^{2k}}{L\alpha} \;=\;  
\frac{1 - (L\alpha-1)^{2k}}{L\alpha} 
$$

$$ 
\begin{aligned}
Sub_2 
&= -\left[-(L\alpha - 1)^3 + (L\alpha - 1)^2 - (L\alpha - 1) + 1\right]  \\
&= -S \\
&= -\left(\frac{1 - (L\alpha-1)^{2k}}{L\alpha}\right) \\
&= \frac{(L\alpha-1)^{2k} - 1}{L\alpha} \\
\end{aligned}
$$

The constant coefficient can now be written as
$$ \gamma + L\beta^2\left(\frac{1 - (L\alpha-1)^{2k}}{L\alpha}\right) \;=\; \gamma + \beta^2\left(\frac{(L\alpha-1)^{2k}-1}{\alpha}\right) $$

### Putting it together


$$
\begin{aligned}
Loss_k 
&= w^2\left[\alpha(L\alpha - 1)^{2k}\right]+ w\left[-2\beta(L\alpha-1)^{2k}\right]+ \left[\gamma + \beta^2\left(\frac{(L\alpha-1)^{2k}-1}{\alpha}\right)\right] \\
&= w^2\left[\alpha(L\alpha - 1)^{2k}\right]+ w\left[-2\beta(L\alpha-1)^{2k}\right] + \frac{\beta^2(L\alpha-1)^{2k}}{\alpha} + \frac{\alpha\gamma - \beta^2}{\alpha} \\
&= (L\alpha - 1)^{2k}\frac{\alpha^2w^2 - 2\alpha\beta + \beta^2}{\alpha} + \frac{\alpha\gamma - \beta^2}{\alpha} \\
&= (L\alpha - 1)^{2k}\frac{(w\alpha-\beta)^2}{\alpha} + \frac{\alpha\gamma - \beta^2}{\alpha} \\
\end{aligned}
$$


So our final expression
$$
Loss_k = (L\alpha - 1)^{2k}\frac{(w\alpha-\beta)^2}{\alpha} + \frac{\alpha\gamma - \beta^2}{\alpha}
$$

The expanded form of this expression:
$$
Loss_k = \frac{\left(\left(L \sum_{i=0}^{n} {x}_{i}^{2} - 1\right)^{2 k}\right) \left(w \sum_{i=0}^{n} {x}_{i}^{2} - \sum_{i=0}^{n} {x}_{i} {y}_{i}\right)^{2} - \left(\sum_{i=0}^{n} {x}_{i} {y}_{i}\right)^{2} + \left(\sum_{i=0}^{n} {x}_{i}^{2}\right) \sum_{i=0}^{n} {y}_{i}^{2}}{\sum_{i=0}^{n} {x}_{i}^{2}}
$$

Doing a similar analysis for modelling $w$in terms of$k$ gives us

$$w_k = \frac{\beta + (1-L\alpha)^k(w\alpha-b)}{\alpha}$$

#### Analysis

- Loss is an exponential function in terms of $k$; and a quadratic function in terms of $w$.
    - It does not matter what $w$ you start with, the first term rapidly becomes influenced by $k$
    - If $| L\alpha - 1 | > 1$, the loss will explode. Regression would not converge.
- $\frac{\alpha\gamma - \beta^2}{\alpha}$ is constant for a given sample, it does not matter on either $k$, $L$or$w$, I will call this $Loss_{base}$. Your loss cannot be lesser than this.
- $w_k$converges to$\beta / \alpha$(this is the true value, we also come to this exact formula for the true value if we solve for$G=0$)
- If $L\alpha - 1 = 0$, the loss becomes minimum after the first step, no matter which $w$ you start from.
    - The cool thing is that this is a constant and does not depend on $w$.
    - I believe this is because gradient itself is a linear function directly proportional to $w$

##### Cauchy-Schwarz Inequality

Let's look at the loss function
$$ Loss_k = (L\alpha - 1)^{2k}\frac{(w\alpha-\beta)^2}{\alpha} + \frac{\alpha\gamma - \beta^2}{\alpha} $$

$L$here is a free variable, we will look at the special case of$L = \frac{1}{\alpha}$ (the first term becomes zero)

$$ Loss_k = \frac{\alpha\gamma - \beta^2}{\alpha} $$

The original definition of loss from which we derived the above function is
$$ loss = \sum_{i=1}^{n} (y_i - mx_i)^2  $$ 

These two functions are equivalent for some $m$ 
$$ Loss_k = \frac{\alpha\gamma - \beta^2}{\alpha} = \sum_{i=1}^{n} (y_i - m_k x_i)^2 > 0 $$

$$ \frac{\alpha\gamma - \beta^2}{\alpha} > 0 $$
$$ \alpha\gamma > \beta^2 $$
$$ \sum_{i=0}^{n} {x}_{i}^{2} \sum_{i=0}^{n} {y}_{i}^{2} > \left(\sum_{i=0}^{n} {x}_{i} {y}_{i}\right)^{2} $$

This seems to be an indirect proof of [Cauchy-Schwarz](https://artofproblemsolving.com/wiki/index.php/Cauchy-Schwarz_Inequality) Inequality in the elemantary algebraic form. 

## Verification

I might be able to definitively prove this expression using induction. For now however, I'm going to expand my own expression using `sympy` and check if it is the same as the original expression.  

I've also computationally verified this. I do raw gradient descent and see if the losses are matching. I'm not going to do it here.  

In [10]:
# using sympy

def theory_loss_expr(k):
    sub = (L*alpha - 1)**(2*k)
    # expr = w*w*(alpha*sub) - w*(2*beta*sub) + (beta*beta/alpha)*(sub-1) + gamma
    expr = (sub*((w*alpha - beta)**2)) + (alpha*gamma - beta*beta)
    expr /= alpha
    return expr

# check if all losses are the same
# this takes a few seconds to run
for i, l in enumerate(losses):
    if expand(theory_loss_expr(i)) != expand(l):
        raise